<a href="https://colab.research.google.com/github/hrbolek/func2pipe/blob/master/notebooks/func2pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Func to Pipe

## Source Code

### Functions Reducer

In [28]:
def createPipe(*F):
  """Create composition of array of functions.

  Args:
    F: array of functions to be composed

  Returns:
    function which parameter is an iterator and which is the composition of F
  """  
  def Fdiamond(sequence):
    result = sequence
    for Fx in F:
      result = Fx(result)
    return result
  return Fdiamond

### Subpipe

In [29]:
def arrayReducer(sequence):
  """Reduces sequence into array

  Args:
    sequence (Iterator[any]): iterator to be processed

  Returns:
    Array[any]: array which contains all elements from given iterator
  """
  accumulator = []
  f = lambda accumulator, item: [*accumulator, item]
  return reduce(f, sequence, accumulator)

def createSub(assign = lambda source, result: {**source, **result}, reducer = None):
  """Creates function which compose functions handling iterator. 
  
  Args:
    assign : defines how the item on input is combined with its result(s). 
      By default, it combines them as they are dictionaries.
    reducer : if any it packs all results from one item on input into single value. 

  Returns:
    Function which compose given functions to produce compound function operating on iterators.
  """
  def multipleResults(*F):
      buffer = {'data': None}
      def pre(gen):
          for item in gen:
              buffer['data'] = item
              yield item

      def post(gen):
          for item in gen:
              yield assign(buffer['data'], item)

      wholePipe = createPipe(pre, *F, post)
      return wholePipe

  def reducedResult(*F):
      inner = createPipe(*F, reducer)
      def result(gen):
          for item in gen:
              yield inner([item])

      wholePipe = multipleResults(result)
      return wholePipe

  if reducer == None:
      return multipleResults
  else:
      return reducedResult

### Convertor

In [30]:
from functools import wraps, reduce, partial


In [31]:
from functools import wraps, reduce, partial
def convertToPipeFuncFull(func, with_yield = False, with_state = False, kwargs = {}):
  """Allows converting a function in function which operates on iterators.

  Args:
    func: function to be converted.
    with_yield (bool): True if func is generator, otherwise False. 
    with_state (bool): True if func has arg named state and which acts as a state value, otherwise False
    kwargs: defines other values given to func

  Returns:
    Function which operates on iterators.
  """
  @wraps(func)
  def innerSelectSimple(generator):
      for i in generator:
          yield func(i, **kwargs)

  if (not with_yield) & (not with_state):
      return innerSelectSimple
  
  @wraps(func)
  def innerSelectWithYield(generator):
      for i in generator:
          for j in func(i, **kwargs):
              yield j


  if (with_yield) & (not with_state):
      return innerSelectWithYield

  @wraps(func)
  def innerSelectWithState(generator):
      state = None
      for i in generator:
          result = None
          if (state):
              result, state = func(i, state = state, **kwargs)
          else:
              result, state = func(i, **kwargs)
          if result:
              yield result

  if (not with_yield) & with_state:
      return innerSelectWithState

  @wraps(func)
  def innerSelectWithYieldWithState(generator):
      state = None
      initState = True
      for i in generator:
          result = None
          if (initState):
              for result, statel in func(i, **kwargs):
                  state = statel
                  if result:
                      yield result
              initState = False
          else:
              for result, statel in func(i, state = state, **kwargs):
                  state = statel
                  if result:
                      yield result


  return innerSelectWithYieldWithState

### Convertor Decorator

In [32]:
def convertToPipeFunc(with_yield = False, with_state = False):
  """It is decorator which use functionality of convertToPipeFuncFull function.

  Args:
    with_yield (bool): True if the decorated function is generator, otherwise False. 
    with_state (bool): True if the decorated function has arg named state and which acts as a state value, otherwise False
  """

  def pipeit(__func):
      @wraps(__func)
      def binder(**kwargs):
          result = wraps(__func)(convertToPipeFuncFull(__func, with_yield, with_state, kwargs))
          return result
      return binder
  return pipeit

convertToPipeFunc2 = convertToPipeFunc 

def convertLambdaToPipeFunc(lambdaFunc, with_state = False, with_yield = False):
  """It "decorate" lambda function in same way as convertToPipeFunc function

  Args:
    lambdaFunc : function to be decorated
    with_yield (bool): True if the decorated function is generator, otherwise False. 
    with_state (bool): True if the decorated function has arg named state and which acts as a state value, otherwise False
  """
  result = convertToPipeFunc(with_yield = with_yield, with_state = with_state)(lambdaFunc)
  return result

## Looper

In [33]:
def looper(**kwargs):
  for key in kwargs:
    firstkey = key
    func = kwargs[firstkey]
    if not key == 'with_yield':
      break
  def wrapper(f):
    def withStartValue(value):
      def proc(generator):
        cvalue = value
        for item in generator:
          result = f(item, **{firstkey: cvalue})
          cvalue = func(result)
          yield result
      return proc
    return withStartValue
  return wrapper

@looper(index = lambda item: item['index'])
def simple(item, index):
  return {**item, 'index': index + 1}

gen = [{'v': i} for i in range(4)]
#operator
for j in simple(0)(gen):
  print(j)

{'v': 0, 'index': 1}
{'v': 1, 'index': 2}
{'v': 2, 'index': 3}
{'v': 3, 'index': 4}


### Ver 2

In [56]:
def looper(with_yield = False, **kwargs):
  for key in kwargs:
    firstkey = key
    func = kwargs[firstkey]
    if not key == 'with_yield':
      break
    else:
      print('with_yield')
  print(kwargs)

  def prepareParams(kw, item=None):
    result = {**kw}
    if not item is None:
      for key in kwargs:
        result[key] = kwargs[key](item)
    return result

  def wrapper(f):
    def binder(**kwargs2):
      def proc(generator):
        cvalue = kwargs2[firstkey]
        for item in generator:
          result = f(item, **{**kwargs2, firstkey: cvalue})
          cvalue = func(result)
          yield result

      def procWY(generator):
        cvalue = kwargs2[firstkey]
        for item in generator:
          result = f(item, **{**kwargs2, firstkey: cvalue})
          for subresult in result:
            cvalue = func(subresult)
            yield subresult

      if with_yield:
        return procWY
      else:
        return proc

    return binder
  return wrapper

@looper(with_yield=False, index = lambda item: item['index'])
def simple(item, index):
  return {**item, 'index': index + 1}

gen = [{'v': i} for i in range(4)]

operator = createPipe(
    simple(index=0),
    simple(index=0),
)
for j in operator(gen):
  print(j)

{'index': <function <lambda> at 0x7ff626d31e18>}
{'v': 0, 'index': 1}
{'v': 1, 'index': 2}
{'v': 2, 'index': 3}
{'v': 3, 'index': 4}


### Ver 3

In [81]:
def looperFull(f, funcDict, valueDict, with_yield = False):
  for key in funcDict:
    firstkey = key
    func = funcDict[firstkey]
    #if not key == 'with_yield':
    break

  def prepareParams(kw, item=None):
    result = {**kw}
    if not item is None:
      for key in funcDict:
        result[key] = funcDict[key](item)
    return result

  def proc(generator):
    params = {**valueDict}
    for item in generator:
      result = f(item, **params)
      params = prepareParams(valueDict, result)
      yield result

  def procWY(generator):
    params = {**valueDict}
    for item in generator:
      result = f(item, **params)
      for subresult in result:
        yield subresult
      params = prepareParams(valueDict, subresult)

  if with_yield:
    return procWY
  else:
    return proc

def looperFullReduced(with_yield = False, **funcDict):
  def child(f):
    @wraps(f)
    def grandChild(**valueDict):
      result = looperFull(funcDict, f, valueDict, with_yield=with_yield)
      return wraps(f)(result)
    return grandChild
  return child

def looper(with_yield = False, **kwargs):
  for key in kwargs:
    firstkey = key
    func = kwargs[firstkey]
    if not key == 'with_yield':
      break
    else:
      #print('with_yield')
      pass
  #print(kwargs)

  def prepareParams(kw, item=None):
    result = {**kw}
    if not item is None:
      for key in kwargs:
        result[key] = kwargs[key](item)
    return result

  def wrapper(f):
    def binder(**kwargs2):

      def proc(generator):
        params = {**kwargs2}
        for item in generator:
          result = f(item, **params)
          params = prepareParams(kwargs2, result)
          yield result

      def procWY(generator):
        params = {**kwargs2}
        for item in generator:
          result = f(item, **params)
          for subresult in result:
            yield subresult
          params = prepareParams(kwargs2, subresult)

      if with_yield:
        return procWY
      else:
        return proc

    return binder
  return wrapper

@looperFullReduced(with_yield=False, index = lambda item: item['index'])
def simple(item, index):
  return {**item, 'index': index + 1}

@looperFullReduced(with_yield=True, index = lambda item: item['index'])
def simpleB(item, index):
  yield {**item, 'index': index + 1, 'ver': 'A'}
  yield {**item, 'index': index + 1, 'ver': 'B'}

@looperFullReduced(with_yield=False, 
  index = lambda item: item['index'],
  value2 = lambda item: item['v'])
def notSoSimple(item, index, value2):
  return {**item, 'notSoSimple': index + value2}

gen = ({'v': i} for i in range(4))

operator = createPipe(
    simpleB(index=0),
    notSoSimple(index=0, value2=1),
)
for j in operator(gen):
  print(j)

{'v': 0, 'index': 1, 'ver': 'A', 'notSoSimple': 1}
{'v': 0, 'index': 1, 'ver': 'B', 'notSoSimple': 1}
{'v': 1, 'index': 2, 'ver': 'A', 'notSoSimple': 1}
{'v': 1, 'index': 2, 'ver': 'B', 'notSoSimple': 3}
{'v': 2, 'index': 3, 'ver': 'A', 'notSoSimple': 3}
{'v': 2, 'index': 3, 'ver': 'B', 'notSoSimple': 5}
{'v': 3, 'index': 4, 'ver': 'A', 'notSoSimple': 5}
{'v': 3, 'index': 4, 'ver': 'B', 'notSoSimple': 7}


In [69]:
@looper()
def systemFilter(item, gen):
  return {**item, 'state': next(gen)}

gen = ({'t': i/100} for i in range(4))
operator = createPipe(
    systemFilter(gen=iter(range(1000))),
)
for j in operator(gen):
  print(j)

{}
{'t': 0.0, 'state': 0}
{'t': 0.01, 'state': 1}
{'t': 0.02, 'state': 2}
{'t': 0.03, 'state': 3}


## Tests

In [35]:
import unittest

class TestCase(unittest.TestCase):
  def __call__(self, *args, **kwargs):
    self.assertEqual(*args, **kwargs)
    return True

testEquality = TestCase()

### Simple Function

In [84]:
def test_simpleFunction(printEnabled = False):
  source = [{'value': 0}, {'value': 1}, {'value': 2}]
  @looperFullReduced()
  def add(item, amount):
    return {**item, 'result': item['value'] + amount}

  expectedResult = [{'value': 0, 'result': 3}, {'value': 1, 'result': 4},
    {'value': 2, 'result': 5}]

  pipe = createPipe(
      add(amount = 2),
      add(amount = 3),
      list)

  result = pipe(source)
  equality = testEquality(expectedResult, result)
  if printEnabled:
    print('Source:', source)
    print('Result:', result)
    print('Valid: ', equality) 

test_simpleFunction()    

### Function with Yield

In [85]:
def test_functionWithYield(printEnabled = False):
  source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

  @looperFullReduced(with_yield = True)
  def revealSubItem(item, itemName):
      for _ in item[itemName]:
          yield _

  expectedResult = ['A', 'B', 'A', 'C', 'D', 'E']

  pipe = createPipe(
      revealSubItem(itemName = 'value'),
      list)

  result = pipe(source)
  equality = testEquality(expectedResult, result)
  if printEnabled:
    print('Source:', source)
    print('Result:', result)
    print('Valid: ', equality) 

test_functionWithYield()  

### Function with State

In [38]:
def test_functionWithState(printEnabled = False):
  source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

  @convertToPipeFunc(with_state = True)
  def assignId(item, state = 0, idName = 'id'):
      return {**item, idName: state}, state + 1

  expectedResult = [{'value': ['A', 'B'], 'ID': 0}, 
      {'value': ['A', 'C'], 'ID': 1}, {'value': ['D', 'E'], 'ID': 2}]

  pipe = createPipe(
      assignId(idName = 'ID'),
      list)

  result = pipe(source)
  equality = testEquality(expectedResult, result)
  if printEnabled:
    print('Source:', source)
    print('Result:', result)
    print('Valid: ', equality) 

test_functionWithState()

### Function with Yield and with State

In [39]:
def test_functionWithBoth(printEnabled = False):
  source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

  @convertToPipeFunc(with_yield = True, with_state = True)
  def assignIdToSubItem(item, state = -1, itemName = ''):
      for _ in item[itemName]:
          state = state + 1
          yield {itemName: _, 'id': state}, state

  expectedResult = [{'value': 'A', 'id': 0}, {'value': 'B', 'id': 1}, 
    {'value': 'A', 'id': 2}, {'value': 'C', 'id': 3}, 
    {'value': 'D', 'id': 4}, {'value': 'E', 'id': 5}
    ]

  pipe = createPipe(
      assignIdToSubItem(itemName = 'value'),
      list)

  result = pipe(source)
  equality = testEquality(expectedResult, result)
  if printEnabled:
    print('Source:', source)
    print('Result:', result)
    print('Valid: ', equality) 

test_functionWithBoth()

### Subpipe

In [40]:
def test_functionSubPipe(printEnabled = False):
  source = [{'value': 0}, {'value': 1}, {'value': 2}]
  @convertToPipeFunc()
  def plus(item, amount):
    return item + amount

  @convertToPipeFunc()
  def selectIt(item, f):
    return f(item)

  expectedResult = [
    {'value': 0, 'newvalue': 2}, 
    {'value': 1, 'newvalue': 3}, 
    {'value': 2, 'newvalue': 4}
    ]

  pipe = createPipe(
      createSub(assign = lambda source, result: {**source, 'newvalue': result})(
        selectIt(f = lambda item: item['value']),
        plus(amount = 2),
      ),
      list)

  result = pipe(source)
  equality = testEquality(expectedResult, result)
  if printEnabled:
    print('Source:', source)
    print('Result:', result)
    print('Valid: ', equality) 

test_functionSubPipe()

### Subpipe II

In [41]:
def test_functionSubPipeII(printEnabled = False):
  source = [{'values': [0, 1]}, {'values': [2, 1]}, {'values': [5, 2]}]
  @convertToPipeFunc()
  def plus(item, amount):
      return item + amount

  @convertToPipeFunc(with_yield=True)
  def selectItAndEnum(item, f):
      for _ in f(item):
        yield _

  def createReducer():
      def reducer(source):
          total = 0
          for item in source:
            total = total + item
          return total 
      return reducer

  expectedResult = [
    {'values': [0, 1], 'sum': 1}, 
    {'values': [2, 1], 'sum': 3}, 
    {'values': [5, 2], 'sum': 7}
    ]

  pipe = createPipe(
      createSub(assign = lambda source, result: {**source, 'sum': result},
        reducer = createReducer())(
        selectItAndEnum(f = lambda item: item['values']),
      ),
      list)

  result = pipe(source)
  equality = testEquality(expectedResult, result)
  if printEnabled:
    print('Source:', source)
    print('Result:', result)
    print('Valid: ', equality) 

test_functionSubPipeII()  

In [42]:
source = [{'value': 0}, {'value': 1}, {'value': 2}]
@convertToPipeFunc()
def add(item, amount):
  return {**item, 'result': item['value'] + amount}

@convertToPipeFunc()
def calc(item, func, name):
  result = {}
  result[name] = func(item)
  return result

@convertToPipeFunc(with_yield = True)
def extra(item, extradata):
  for _ in extradata:
    yield _

def createReducer(accumulator = [], f = lambda item, accumulator: [*accumulator, item]):
  def reducer(gen):
    result = accumulator
    for item in gen:
      result = f(item, result)
    return result
  return reducer

pipe_01 = createPipe(
    add(amount = 2),
    createSub(assign = lambda source, result: {**source, 'value2': result}, reducer = createReducer())(
#    SUB(assign = lambda source, result: {**source, **result})(
#    SUB(assign = lambda source, result: {**source, 'value2': result})(
      calc(func = lambda item: 4, name = 'extraValue'),
      extra(extradata = ['A', 'B', 'C'])
    )
    )
result = list(pipe_01(source))
print(result)


[{'value': 0, 'result': 2, 'value2': ['A', 'B', 'C']}, {'value': 1, 'result': 3, 'value2': ['A', 'B', 'C']}, {'value': 2, 'result': 4, 'value2': ['A', 'B', 'C']}]


## Pipe Described by Graph 

In [43]:
def createNumericAggregator(startValue = 0, aggFunc = lambda x, y: x + y):
  """creates functions which operates on iterator and which returns reduced value.

  Args:
    startValue: initial value of accumulator
    aggFunc: function which reduce accumulator value and value from iterator

  Returns:
    function operating on iterator
  """
  def inner(gen):
    result = startValue
    for item in gen:
      result = aggFunc(result, item)
    return result
  return inner

def createArrayReducer(accumulator = [], f = lambda accumulator, item: [*accumulator, item]):
  def reducer(gen):
    return reduce(f, gen, accumulator)
  return reducer

@convertToPipeFunc(with_yield=True)
def stopIfNotFound(item):
  if not item is None:
    yield item

@convertToPipeFunc(with_yield=True)
def filterIt(item, filterFunc):
  if filterFunc(item):
    yield item

@convertToPipeFunc(with_yield=True)
def selectSubItems(item, selectFunc):
  result = selectFunc(item)
  if not result is None:
    for item in result:
      yield item

def graphToPipe(graph, currentnode):
  """returns function operating on iterator defined by graph description

  Args:
    graph: data structure describing data sources (nodes), how to reach them (functions) and relations between them (edges) and how to derive them (functions)
    currentnode: node which determines main data source

  Returns:
    function operating on iterator and returning complex dictionary containing extended data
  """
  availableNodes = []
  for node in graph['nodes']:
    availableNodes.append(node)

  def createAssignLambda(itemName):
    return lambda item, result: {**item, itemName: result}

  def reducersIntoPipes(relation):
    result = []
    for reduction in relation['reducers']:
      reSub = createSub(
          assign = createAssignLambda(reduction['name']),
          reducer = reduction['reduce'])
      result.append(reSub(
          selectSubItems(selectFunc = lambda item: item[relation['itemname'] if 'itemname' in relation else relation['to']]),
          reduction['map']))
    return result

  def innerBuilder(graph, currentnode, filterq = None):
      availableNodes.remove(currentnode)
      descriptorPipe = graph['nodes'][currentnode]

      def buildRelation(relation):
        itemname = relation['itemname'] if 'itemname' in relation else relation['to']
        filterq = relation['filter'] if 'filter' in relation else lambda item: True

        sub = createSub(assign = createAssignLambda(itemname), reducer = createArrayReducer())
        subPipe = innerBuilder(graph, relation['to'], filterq)
        result = [sub(relation['relation'], *subPipe)]
        if 'reducers' in relation:
          pipes = reducersIntoPipes(relation)
          result.extend(pipes)

        return result

      result = [descriptorPipe, stopIfNotFound()]
      if filterq:
        result.append(filterIt(filterFunc = filterq))
      
      nodesToRemove = []
      for relation in filter(lambda item: item['from'] == currentnode, graph['edges']):
          nodeName = relation['to']
          nodesToRemove.append(nodeName)
          savedAvaliableNodes = availableNodes[:] #copy state
          if nodeName in savedAvaliableNodes:
            result += [*buildRelation(relation)]
          availableNodes.clear() #restore state
          availableNodes.extend(savedAvaliableNodes)  #restore state II
          
      for node in nodesToRemove:
        if node in availableNodes:
          availableNodes.remove(node)

      return result
  result = innerBuilder(graph, currentnode)
  return createPipe(*result)

### Database Model

In [44]:
studentTable = [
    {'id': 258, 'name': 'George Burden', 'longlifeeducation': False},
    {'id': 263, 'name': 'Julia Seven', 'longlifeeducation': False},
    {'id': 396, 'name': 'Anthony Previous', 'longlifeeducation': True},
]

subjectTable = [
    {'id': 1024, 'name': 'Mathematics', 'lessons': 60},
    {'id': 1144, 'name': 'English', 'lessons': 30},
    {'id': 1194, 'name': 'History', 'lessons': 75},
    {'id': 1086, 'name': 'Physics', 'lessons': 45},
]

student2SubjectTable = [
    {'id': 1, 'studentId': 258, 'subjectId': 1024},
    {'id': 2, 'studentId': 258, 'subjectId': 1086},
    {'id': 3, 'studentId': 263, 'subjectId': 1144},
    {'id': 4, 'studentId': 263, 'subjectId': 1198},
    {'id': 5, 'studentId': 396, 'subjectId': 1024},
    {'id': 6, 'studentId': 396, 'subjectId': 1144},
]

teacherTable = [
    {'id': 2573, 'name': 'Paul Coleman'},
    {'id': 3168, 'name': 'Igor Mashevic'},
    {'id': 1934, 'name': 'Alex Moon'},
    {'id': 2379, 'name': 'Julia Newman'},
]

subject2Teacher = [
    {'id': 1, 'teacherId': 2573, 'subjectId': 1024},
    {'id': 2, 'teacherId': 2573, 'subjectId': 1086},
    {'id': 3, 'teacherId': 3168, 'subjectId': 1144},
    {'id': 4, 'teacherId': 3168, 'subjectId': 1194},
    {'id': 5, 'teacherId': 1934, 'subjectId': 1144},
    {'id': 6, 'teacherId': 2379, 'subjectId': 1086},
]

def SelectCommand(fromTable, whereLambda):
    for item in filter(whereLambda, fromTable):
        yield item

def SelectWhereId(fromTable, idValue):
  for item in SelectCommand(fromTable, lambda item: item['id'] == idValue):
      return item
  return None

### Database Descriptions as Graph

In [45]:
@convertToPipeFunc()
def getStudentRecord(id):
    return SelectWhereId(fromTable=studentTable, idValue=id)

@convertToPipeFunc()
def getTeacherRecord(id):
    return SelectWhereId(fromTable=teacherTable, idValue=id)

@convertToPipeFunc()
def getSubjectRecord(id):
    return SelectWhereId(fromTable=subjectTable, idValue=id)

@convertToPipeFunc(with_yield = True)
def StudentToSubject(item):
  id = item['id']
  for _ in SelectCommand(student2SubjectTable, lambda record: record['studentId'] == id):
      yield _['subjectId']

@convertToPipeFunc(with_yield = True)
def SubjectToStudent(item):
  id = item['id']
  for _ in SelectCommand(student2SubjectTable, lambda record: record['subjectId'] == id):
      yield _['studentId']

@convertToPipeFunc(with_yield = True)
def SubjectToTeacher(item):
  id = item['id']
  for _ in SelectCommand(subject2Teacher, lambda record: record['subjectId'] == id):
      yield _['teacherId']

@convertToPipeFunc(with_yield = True)
def TeacherToSubject(item):
  id = item['id']
  for _ in SelectCommand(subject2Teacher, lambda record: record['teacherId'] == id):
      yield _['subjectId']

@convertToPipeFunc()
def getLessons(item):
  return item['lessons']


### Usage

In [46]:
import json

grDefinition = {
    'nodes': {
        'Students': getStudentRecord(),
        'Teachers': getTeacherRecord(),
        'Subjects': getSubjectRecord(),
        },
    'edges': [
        {'from': 'Students', 'to': 'Subjects', 'relation': StudentToSubject()},
        {'from': 'Subjects', 'to': 'Students', 'relation': SubjectToStudent()},

        {'from': 'Teachers', 'to': 'Subjects', 'relation': TeacherToSubject()},
        {'from': 'Subjects', 'to': 'Teachers', 'relation': SubjectToTeacher()},
    ]
}

grDefinition = {
    'nodes': {
        'Students': getStudentRecord(),
        'Teachers': getTeacherRecord(),
        'Subjects': getSubjectRecord(),
        },
    'edges': [
        {'from': 'Students', 'to': 'Subjects', 'relation': StudentToSubject(),
         'reducers' :[
            {'name': 'lessons', 
             'map': getLessons(), 
             'reduce': createNumericAggregator(startValue = 0, aggFunc = lambda x, y: (x + y))
             },
          ]
        },
        {'from': 'Subjects', 'to': 'Students', 'relation': SubjectToStudent(),
         'itemname' : 'LongLifeStudents', 
         'filter': lambda item: item['longlifeeducation']
         },
        {'from': 'Subjects', 'to': 'Students', 'relation': SubjectToStudent(),
         'itemname' : 'OrdinaryStudents', 
         'filter': lambda item: not item['longlifeeducation']
         },
        {'from': 'Subjects', 'to': 'Students', 'relation': SubjectToStudent()},

        {'from': 'Teachers', 'to': 'Subjects', 'relation': TeacherToSubject()},
        {'from': 'Subjects', 'to': 'Teachers', 'relation': SubjectToTeacher()},
    ]
}


queryStudents = graphToPipe(grDefinition, 'Students')
queryTeachers = graphToPipe(grDefinition, 'Teachers')
querySubjects = graphToPipe(grDefinition, 'Subjects')

studentsIds = [259, 258]
queryStudentsResult = list(queryStudents(studentsIds))
print(json.dumps(queryStudentsResult, indent=2))

[
  {
    "id": 258,
    "name": "George Burden",
    "longlifeeducation": false,
    "Subjects": [
      {
        "id": 1024,
        "name": "Mathematics",
        "lessons": 60,
        "Teachers": [
          {
            "id": 2573,
            "name": "Paul Coleman"
          }
        ]
      },
      {
        "id": 1086,
        "name": "Physics",
        "lessons": 45,
        "Teachers": [
          {
            "id": 2573,
            "name": "Paul Coleman"
          },
          {
            "id": 2379,
            "name": "Julia Newman"
          }
        ]
      }
    ],
    "lessons": 105
  }
]


### Usage II

In [47]:
studentsIds = [258, 396]
queryStudentsResult = list(queryStudents(studentsIds))
print(json.dumps(queryStudentsResult, indent=2))

[
  {
    "id": 258,
    "name": "George Burden",
    "longlifeeducation": false,
    "Subjects": [
      {
        "id": 1024,
        "name": "Mathematics",
        "lessons": 60,
        "Teachers": [
          {
            "id": 2573,
            "name": "Paul Coleman"
          }
        ]
      },
      {
        "id": 1086,
        "name": "Physics",
        "lessons": 45,
        "Teachers": [
          {
            "id": 2573,
            "name": "Paul Coleman"
          },
          {
            "id": 2379,
            "name": "Julia Newman"
          }
        ]
      }
    ],
    "lessons": 105
  },
  {
    "id": 396,
    "name": "Anthony Previous",
    "longlifeeducation": true,
    "Subjects": [
      {
        "id": 1024,
        "name": "Mathematics",
        "lessons": 60,
        "Teachers": [
          {
            "id": 2573,
            "name": "Paul Coleman"
          }
        ]
      },
      {
        "id": 1144,
        "name": "English",
        "lessons"

### Usage III (Teachers)

In [48]:
teachersIds = [2573]
queryTeachersResult = list(queryTeachers(teachersIds))
print(json.dumps(queryTeachersResult, indent=2))

[
  {
    "id": 2573,
    "name": "Paul Coleman",
    "Subjects": [
      {
        "id": 1024,
        "name": "Mathematics",
        "lessons": 60,
        "LongLifeStudents": [
          {
            "id": 396,
            "name": "Anthony Previous",
            "longlifeeducation": true
          }
        ],
        "OrdinaryStudents": [
          {
            "id": 258,
            "name": "George Burden",
            "longlifeeducation": false
          }
        ],
        "Students": [
          {
            "id": 258,
            "name": "George Burden",
            "longlifeeducation": false
          },
          {
            "id": 396,
            "name": "Anthony Previous",
            "longlifeeducation": true
          }
        ]
      },
      {
        "id": 1086,
        "name": "Physics",
        "lessons": 45,
        "LongLifeStudents": [],
        "OrdinaryStudents": [
          {
            "id": 258,
            "name": "George Burden",
            "longl

### Usage IV (Subjects)

In [49]:
subjectsIds = [1144, 1024]
querySubjectsResult = list(querySubjects(subjectsIds))
print(json.dumps(querySubjectsResult, indent=2))

[
  {
    "id": 1144,
    "name": "English",
    "lessons": 30,
    "LongLifeStudents": [
      {
        "id": 396,
        "name": "Anthony Previous",
        "longlifeeducation": true
      }
    ],
    "OrdinaryStudents": [
      {
        "id": 263,
        "name": "Julia Seven",
        "longlifeeducation": false
      }
    ],
    "Students": [
      {
        "id": 263,
        "name": "Julia Seven",
        "longlifeeducation": false
      },
      {
        "id": 396,
        "name": "Anthony Previous",
        "longlifeeducation": true
      }
    ],
    "Teachers": [
      {
        "id": 3168,
        "name": "Igor Mashevic"
      },
      {
        "id": 1934,
        "name": "Alex Moon"
      }
    ]
  },
  {
    "id": 1024,
    "name": "Mathematics",
    "lessons": 60,
    "LongLifeStudents": [
      {
        "id": 396,
        "name": "Anthony Previous",
        "longlifeeducation": true
      }
    ],
    "OrdinaryStudents": [
      {
        "id": 258,
        "name

### Artificial Experiment

In [50]:

greatSource = [{'id': 'A'}, {'id': 'B'}, {'id': 'C'}]
smallSource = [{'id': 'a'}, {'id': 'b'}, {'id': 'c'}]
fromGreat2Small = {'A': 'a', 'B': 'b', 'C': 'c'}
fromSmall2Great = {'a': 'A', 'b': 'B', 'c': 'C'}

@convertToPipeFunc2()
def greatToSmallRelation(greatItem):
  print('G:', greatItem)
  return {'idS': fromGreat2Small[greatItem['id']]}

@convertToPipeFunc2()
def smallToGreatRelation(smallItem):
  #print('S:', smallItem)
  return {'id': fromSmall2Great[smallItem['id']]}

@convertToPipeFunc2(with_yield=True)
def smallToGreatRelation2(smallItem):
  #print('S:', smallItem)
  for i in greatSource:
    yield i
  #return {'idG': fromSmall2Great[smallItem['id']]}

@convertToPipeFunc2()
def identityPipe(item, description):
  return {**item, 'description': description}

@convertToPipeFunc2(with_yield=True)
def anyToNumbersRelation(item):
  for i in range(1,3):
    #yield {'value': i}
    yield i

@convertToPipeFunc2()
def numberRecord(item):
  return {'value': item}

@convertToPipeFunc2()
def printItem(item, description):
  print(description, item)
  return item

@convertToPipeFunc2()
def getNumbers(item):
  print(item)
  return item['value']
      

gr = {
    'nodes': {
        'Great': identityPipe(description = 'Great item'), 
        'Small': identityPipe(description = 'Small item'),
        'Numbers': numberRecord(),
        },
    'edges': [
        {'from': 'Great', 'to': 'Small', 
         'relation': greatToSmallRelation(), 'itemname' : 'little'},
        {'from': 'Small', 'to': 'Great', 
         'relation': smallToGreatRelation2(), 'itemname' : 'bigA', 
         'filter': lambda item: item['id'] <= 'A'},
        {'from': 'Small', 'to': 'Great', 
         'relation': smallToGreatRelation2(), 'itemname' : 'bigBC', 
         'filter': lambda item: item['id'] > 'A'},
        {'from': 'Small', 'to': 'Numbers', 
         'relation': anyToNumbersRelation(), 'itemname' : 'numbers'},
        {'from': 'Great', 'to': 'Numbers', 
         'relation': anyToNumbersRelation(), 'itemname' : 'numbers',
         'reducers' :[
            {'name': 'count', 
             'map': getNumbers(), 
             'reduce': createNumericAggregator(startValue = 0, aggFunc = lambda x, y: (x + 1))
             },
            {'name': 'sum', 
             'map': getNumbers(), 
             'reduce': createNumericAggregator(startValue = 0, aggFunc = lambda x, y: (x + y))
             },
          ]},
    ]
}

_gr = {
    'nodes': {
        'Great': identityPipe(description = 'Great item'), 
        'Small': identityPipe(description = 'Small item'),
        'Numbers': numberRecord(),
        },
    'edges': [
        {'from': 'Great', 'to': 'Small', 
         'relation': greatToSmallRelation(), 'itemname' : 'little'},
        {'from': 'Small', 'to': 'Great', 
         'relation': smallToGreatRelation2(), 'itemname' : 'bigA', 
         'filter': lambda item: item['id'] <= 'A'},
        {'from': 'Small', 'to': 'Great', 
         'relation': smallToGreatRelation2(), 'itemname' : 'bigBC', 
         'filter': lambda item: item['id'] > 'A'},
        {'from': 'Small', 'to': 'Numbers', 
         'relation': anyToNumbersRelation(), 'itemname' : 'numbers'},
        {'from': 'Great', 'to': 'Numbers', 
         'relation': anyToNumbersRelation(), 'itemname' : 'numbers'},
    ]
}
import json
graphPipe = graphToPipe(gr, 'Small')
result = list(graphPipe(smallSource))
print(json.dumps(result, indent=2))

{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
{'value': 1}
{'value': 2}
[
  {
    "id": "a",
    "description": "Small item",
    "bigA": [
      {
        "id": "A",
        "description": "Great item",
        "numbers": [
          {
            "value": 1
          },
          {
            "value": 2
          }
        ],
        "count": 2,
        "sum": 3
      }
    ],
    "bigBC": [
      {
        "id": "B",
        "description": "Great item",
        "numbers": [
          {
            "value": 1
          },
          {
            "value": 2
          }
        ],
        "count"

## Timeit - Duration measurement

In [51]:
@convertToPipeFunc2()
def copy(item, description):
    """describes and converts item into dictionary."""
    return {'input': item, 'description': description}

@convertToPipeFunc2(with_yield = True)
def copyY(item, description):
    """describes and converts item into dictionary."""
    yield {'input': item, 'description': description}

@convertToPipeFunc2(with_yield = False, with_state = True)
def copyS(item, description, state = 0):
    """describes and converts item into dictionary."""
    return ({'input': item, 'description': description, 'state': state}, state+1)

@convertToPipeFunc2(with_yield = True, with_state = True)
def copyYS(item, description, state = 0):
    """describes and converts item into dictionary."""
    yield ({'input': item, 'description': description, 'state': state}, state+1)

def newcopyY(gen):
  for item in gen:
    yield {'input': item, 'description': 'desc'}

def newcopyYS(gen):
  state = 0
  for item in gen:
    yield {'input': item, 'description': 'desc', 'state': state}
    state = state + 1

print(copy.__doc__)
print(copyY.__doc__)

data = [1, 2, 3]
#%%time
print(list(copyS(description = 'desc')(data)))
print(list(copyYS(description = 'desc')(data)))
data = list(range(0, 100000))
print('normal')
%timeit (list(copy(description = 'desc')(data)))
print('normal + State')
%timeit (list(copyS(description = 'desc')(data)))

print('Yield')
%timeit (list(copyY(description = 'desc')(data)))
print('Yield + State')
%timeit (list(copyYS(description = 'desc')(data)))

print('hardcoded normal')
%timeit (list(newcopyY(data)))
print('hardcoded normal + State')
%timeit (list(newcopyYS(data)))


describes and converts item into dictionary.
describes and converts item into dictionary.
[{'input': 1, 'description': 'desc', 'state': 0}, {'input': 2, 'description': 'desc', 'state': 1}, {'input': 3, 'description': 'desc', 'state': 2}]
[{'input': 1, 'description': 'desc', 'state': 0}, {'input': 2, 'description': 'desc', 'state': 1}, {'input': 3, 'description': 'desc', 'state': 2}]
normal
10 loops, best of 3: 44.3 ms per loop
normal + State
10 loops, best of 3: 77.6 ms per loop
Yield
10 loops, best of 3: 57.1 ms per loop
Yield + State
10 loops, best of 3: 91.6 ms per loop
hardcoded normal
10 loops, best of 3: 29.9 ms per loop
hardcoded normal + State
10 loops, best of 3: 38.3 ms per loop
